In [2]:

import gurobipy as gp
from gurobipy import GRB

In [28]:
categories, minNutrition, maxNutrition = gp.multidict({
    'calories': [1800, 2200],
    'protein':  [91, GRB.INFINITY],
    'fat':      [0, 65],
    'sodium':   [0, 1779]})

foods, cost = gp.multidict({
    'hamburger': 2.49,
    'chicken':   2.89,
    'hot dog':   1.50,
    'fries':     1.89,
    'macaroni':  2.09,
    'pizza':     1.99,
    'salad':     2.49,
    'milk':      0.89,
    'ice cream': 1.59})

In [4]:
# Nutrition values for the foods
nutritionValues = {
    ('hamburger', 'calories'): 410,
    ('hamburger', 'protein'):  24,
    ('hamburger', 'fat'):      26,
    ('hamburger', 'sodium'):   730,
    ('chicken',   'calories'): 420,
    ('chicken',   'protein'):  32,
    ('chicken',   'fat'):      10,
    ('chicken',   'sodium'):   1190,
    ('hot dog',   'calories'): 560,
    ('hot dog',   'protein'):  20,
    ('hot dog',   'fat'):      32,
    ('hot dog',   'sodium'):   1800,
    ('fries',     'calories'): 380,
    ('fries',     'protein'):  4,
    ('fries',     'fat'):      19,
    ('fries',     'sodium'):   270,
    ('macaroni',  'calories'): 320,
    ('macaroni',  'protein'):  12,
    ('macaroni',  'fat'):      10,
    ('macaroni',  'sodium'):   930,
    ('pizza',     'calories'): 320,
    ('pizza',     'protein'):  15,
    ('pizza',     'fat'):      12,
    ('pizza',     'sodium'):   820,
    ('salad',     'calories'): 320,
    ('salad',     'protein'):  31,
    ('salad',     'fat'):      12,
    ('salad',     'sodium'):   1230,
    ('milk',      'calories'): 100,
    ('milk',      'protein'):  8,
    ('milk',      'fat'):      2.5,
    ('milk',      'sodium'):   125,
    ('ice cream', 'calories'): 330,
    ('ice cream', 'protein'):  8,
    ('ice cream', 'fat'):      10,
    ('ice cream', 'sodium'):   180}

In [40]:
# Model
m = gp.Model("diet")
# Create decision variables for the foods to buy
buy = m.addVars(foods, name="buy")
binary = m.addVars(foods, name="binary", vtype=GRB.BINARY)
count = m.addVars(foods, name="count", vtype=GRB.INTEGER)
# Using looping constructs, the preceding statement would be:

m.setObjective(sum(buy[f]*cost[f] for f in foods), GRB.MINIMIZE)

M = 200
eps = 0.001
for f in foods:
    m.addConstr(buy[f] >= eps - M * (1 - binary[f]))
    m.addConstr(buy[f] <= M * binary[f])
    # Add indicator constraints
    m.addConstr((binary[f] == 1) >> (count[f] == 1))
    m.addConstr((binary[f] == 0) >> (count[f] == 0))
    m.update()

lhs = sum(count[f] for f in foods)
m.addConstr(lhs >= 9) 

# Nutrition constraints
m.addConstrs((gp.quicksum(nutritionValues[f, c] * buy[f] for f in foods)
             == [minNutrition[c], maxNutrition[c]]
             for c in categories), "_")


m.update()

# Using looping constructs, the preceding statement would be:
#
# for c in categories:
#  m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods),
#             minNutrition[c], maxNutrition[c], c)


def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nCost: %g' % m.ObjVal)
        print('\nBuy:')
        for f in foods:
            if buy[f].X > 0:
                print('%s %g' % (f, buy[f].X))
    else:
        print('No solution')


# Solve
m.optimize()
m.write("file.lp")
printSolution()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23 rows, 30 columns and 84 nonzeros
Model fingerprint: 0xdeac4578
Model has 18 general constraints
Variable types: 12 continuous, 18 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [1e+00, 2e+03]
  RHS range        [9e+00, 2e+03]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 19 rows and 20 columns
Presolve time: 0.00s
Presolved: 4 rows, 10 columns, 37 nonzeros
Variable types: 10 continuous, 0 integer (0 binary)

Root relaxation: objective 1.183295e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      11.83